# Notebook de Modelagem

Na sequência de notebooks, esse é responsável por conter a parte de modelagem e avaliação de modelos de predição. Aqui defino a estratégia como uma de detecção de anomalias -- que é o que faz sentido. Uma anomalia, pela definição encontrada na literatura, é uma instância de um conjunto de dados que produz um padrão diferente do esperado de forma a levantar suspeitas de que foi gerada por um mecanismo diferente. Tal evento anômalo é de interesse do analista, pois leva a compreender o significado e origem daquele padrão. Nesse problema a anomalia é um cliente fraudulento cujo padrão de comportamento difere do esperado de um cliente regular. Essas diferenças foram vislumbradas por meio das vizualizações de dados. A visão completa de todas as possíveis relações e como definir uma região de comportamento anômala deve ficar a cargo de um modelo de predição, pois fazer isso manualmente é complexo e possivelmente ineficiente.

Irei tentar 5 modelos de predição:
- SVM
- Floresta Aleatória
- Floresta de Isolamento
- Deviation Network

Junto a esse notebook coloco dois resumos de resultados em arquivos .csv: 
- Um deles será os resultados obtidos com a modelagem após toda a manipulação de dados e escolhas de atributos. 
- O segundo será um desvio da modelagem em direção a um problema de detecção de anomalias. Num problema de detecção de fraudes como esse pode ser comum que não se tenha muitos dados rotulados sobre o problema, os dados podem estar contaminados e os modelos gerados geralmente não reagem bem a novos tipos de anomalias. Irei reproduzir o problema da pouca representatividade da classe de anomalia. 

O que se observa na comparação é que os métodos feitos para detectar esses tipos de eventos são os que se saem melhor no problema nesse cenário. A pouca representatividade da classe anômala apresenta um grande problema para os modelos de classificação que até apresentam uma boa precisão, mas a os falsos negativos são muito mais frequentes -- o recall é muito baixo.


## Imports

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report, precision_recall_curve, auc, average_precision_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
import time
import snntorch as snn
from snntorch import utils
from snntorch import spikegen, surrogate



random_state = np.random.RandomState(5)
torch.manual_seed(5)
notebook_start_time = time.time()

## Carregamento dos dados

In [68]:
data = pd.read_csv('dataset_case_boleto_fe.csv', low_memory=False)

## Definição de estruturas e pipelines comuns para todos os modelos

In [69]:
def compose_column_transformer(categorical_features: list[str], numerical_features: list[str]) -> ColumnTransformer:
    """
    Compose a ColumnTransformer with a list of categorical and numerical features.

    :param categorical_features: list of categorical features to include in the ColumnTransformer
    :param numerical_features: list of numerical to include in the ColumnTransformer

    :return ColumnTransformer to be used in the pipeline or to simply transform the dataframe
    """
    
    return ColumnTransformer(
        transformers=[
            ('categorical', OneHotEncoder(handle_unknown='ignore', sparse=False, drop='first'), categorical_features),
            ('numerical', StandardScaler(), numerical_features)
            # ('numerical', RobustScaler(), numerical_features)
        ],
        remainder='drop'
    )


def model_evaluation(y_true, y_pred, model_name: str):
    """
    Print the evaluation metrics for the model and return a dictionary with the metrics.

    :param y_true: true labels
    :param y_pred: predicted probabilities for the positive class
    :param model_name: name of the model

    :return dictionary with the evaluation metrics
    """

    print(f'AUC-ROC: {roc_auc_score(y_true, y_pred):.2f}')
    print(f'AUC-PR: {average_precision_score(y_true, y_pred):.2f}')
    print(f'Precision: {precision_score(y_true, y_pred):.2f}')
    print(f'Recall: {recall_score(y_true, y_pred):.2f}')
    print(f'F1-Score: {f1_score(y_true, y_pred):.2f}')
    print(f'Classification Report:\n{classification_report(y_true, y_pred)}')
    return {
        'model-name': model_name,
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1-score': f1_score(y_true, y_pred),
        'auc-roc': roc_auc_score(y_true, y_pred),
        'auc-pr': average_precision_score(y_true, y_pred),
    }

logbook = []

## Seleção de features

As features comentadas não são usadas no modelo, mas foram consideradas e removidas com base no que foi observado na análise de feature importance do Random Forest.

In [70]:
categorical_features = [
    # 'company_id', 
    # 'mes_ref', 
    # 'tipo_doc', 
    # 'tempo_credenciamento',
    'conta_bnk_repetida', 
    # 'fraude', 
    # 'fraude_bool', 
    # 'nome_mes_ref',
    # 'conta_bnk_repetida_bool',
    ]

numerical_features = [
    'max_valor_boleto',
    'avg_valor_boleto', 
    'total_valor_boleto', 
    # 'valor_boleto_stdv',
    # 'qtd_boleto_pago', 
    # 'qtd_boleto_total', 
    'qtd_boleto_estorno',
    # 'qnt_cc_total', 
    'tempo_credenciamento', 
    'mean_max_valor_boleto', 
    'median_max_valor_boleto',
    # 'std_max_valor_boleto', 
    'mean_total_valor_boleto',
    'median_total_valor_boleto', 
    'std_total_valor_boleto',
    # 'max_qtd_boleto_pago', 
    # 'mean_qtd_boleto_pago',
    'mean_qtd_boleto_estorno', 
    'sum_qtd_boleto_total',
    # 'qtd_boleto_nao_pago', 
    # 'rel_qtd_boleto_nao_pago',
    # 'rel_qtd_boleto_estorno', 
    # 'rel_qtd_boleto_pago',
    # 'freq_rel_cc_total_cc_boleto'
    ]

target_column = 'fraude'

## Modelagem e avaliação

### Separação de variáveis dependentes e independente e do conjunto de teste

`X` vai ser todo o conjunto de variáveis independentes, visto que ColumnTransformer irá filtrar as variáveis que não serão usadas.

In [71]:
X = data.drop([target_column, 'fraude_bool'], axis=1)
y = data[target_column]

Agora a separação do conjunto de treino e teste.

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state, stratify=y)

Nessa próxima célula a última linha remove a maior parte das anomalias do conjunto de treino. Dessa forma o treino se torna uma situação inicial de detecção de fraudes.

In [73]:
def select_positive_class(X_train, y_train, count):
    positive_samples = y_train[y_train == 1].sample(count, random_state=random_state).index
    return X_train.drop(positive_samples, axis=0), y_train.drop(positive_samples, axis=0)

# X_train, y_train = select_positive_class(X_train, y_train, count=1056) # Dessa forma ficam apenas 60 instâncias de fraude no conjunto de treino

### Random Forest

O algoritmo de floresta aleatória demonstra bons resultados em diversos de classificação e regressão. Também é um método não paramétrico. Tendo em mente que as classes estão bem distintas na análise é esperado o bom desempenho mostrado aqui.

Construo o pipeline do modelo e executo um treino inicial.

In [74]:
random_forest_model = Pipeline(steps=[
    ('column_transformer', compose_column_transformer(categorical_features, numerical_features)),
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1))
])

A avaliação do modelo sem busca de hiperparâmetros:

In [75]:
random_forest_model.fit(X_train, y_train)
y_hat = random_forest_model.predict(X_test)
model_evaluation(y_test, y_hat, 'Random Forest');

AUC-ROC: 0.90
AUC-PR: 0.72
Precision: 0.82
Recall: 0.85
F1-Score: 0.83
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      2306
           1       0.82      0.85      0.83       478

    accuracy                           0.94      2784
   macro avg       0.89      0.90      0.90      2784
weighted avg       0.94      0.94      0.94      2784



O modelo apresenta um bom desempenho para a classe positiva, identificando com precisão as anomalias e com poucas sendo deixadas na classe negativa.

Posteriormente realizei uma análise de feature importance para determinar quais features manter. Após várias remoções decidi ficar com essas:

In [76]:
feature_names = random_forest_model[0].get_feature_names_out()
feature_importancs = random_forest_model[1].feature_importances_
pd.DataFrame({'feature': feature_names, 'importance': feature_importancs}).sort_values('importance', ascending=False)

,feature,importance
8,numerical__mean_total_valor_boleto,0.172544
9,numerical__median_total_valor_boleto,0.145899
6,numerical__mean_max_valor_boleto,0.103983
7,numerical__median_max_valor_boleto,0.101910
3,numerical__total_valor_boleto,0.081705
5,numerical__tempo_credenciamento,0.062905
2,numerical__avg_valor_boleto,0.062021
12,numerical__sum_qtd_boleto_total,0.056095
1,numerical__max_valor_boleto,0.053756
0,categorical__conta_bnk_repetida_1,0.051106


Por fim, uma busca por hiperparâmetros para o modelo:

In [77]:
random_forest_model_grid_search = GridSearchCV(
    random_forest_model,
    {
        'random_forest__n_estimators': [100, 200],
        'random_forest__max_depth': [None, 10, 30],
        'random_forest__max_features': [1.0, 0.7, 0.5],     
    },
    scoring=[
        'precision',
        'recall',
        'roc_auc',
        'average_precision',
    ],
    refit='average_precision',
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state),
    n_jobs=1,
    verbose=1,
)

random_forest_model_grid_search.fit(X_train, y_train);

Fitting 3 folds for each of 18 candidates, totalling 54 fits


O resultado tende a melhorar com a busca de hiperparâmetros:

In [78]:
y_hat = random_forest_model_grid_search.predict(X_test)
log =  model_evaluation(y_test, y_hat, 'Random Forest (Grid Search)')
logbook.append(log)

AUC-ROC: 0.91
AUC-PR: 0.71
Precision: 0.80
Recall: 0.86
F1-Score: 0.83
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      2306
           1       0.80      0.86      0.83       478

    accuracy                           0.94      2784
   macro avg       0.89      0.91      0.90      2784
weighted avg       0.94      0.94      0.94      2784



Os melhores parâmetros para o modelo são:

In [79]:
random_forest_model_grid_search.best_params_

{'random_forest__max_depth': 10,
 'random_forest__max_features': 0.7,
 'random_forest__n_estimators': 200}

### Deviation Network

A Deviation Network é uma arquitetura de rede neural que dá um score de anomalia para uma amostra. A otimização força a rede a dar um score alto para desvios do padrão de normalidade para amostras que são anomalias. Essa técnica pode ser considerada um aprendizado semi-supervisionado. Num cenário de poucas amostras anômalas é esperado que o modelo performe bem identificando as anomalias.

Classes e funções utilizadas:

In [80]:

class CaseDataset(Dataset):
    def __init__(self, X: np.array, targets: np.array):
        self.X = X
        self.targets = targets

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        item = self.X[index, :]
        item = torch.from_numpy(item).float()
        targets = self.targets[index]
        targets = torch.from_numpy(np.array(targets)).float()
        return item, targets

class DeviationLoss(nn.Module):
    def __init__(self,):
        super(DeviationLoss, self).__init__()

    def forward(self, outputs, targets):
        # Flatten
        outputs = outputs.view(-1)
        targets = targets.view(-1)

        confidence_margin = 5.
        ref = torch.from_numpy(np.random.normal(
            loc=0., scale=1.0, size=5000)).float()
        dev = (outputs - torch.mean(ref)) / torch.std(ref)
        inlier_loss = torch.abs(dev)
        outlier_loss = torch.abs(
            torch.max(confidence_margin - dev, torch.tensor(0.)))
        loss = torch.mean((1 - targets) * inlier_loss + targets * outlier_loss)
        return loss

class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(input_dim, 20),
            nn.ReLU(),
            nn.Linear(20, 1),
        )

    def forward(self, x):
        return self.stack(x)

def train_loop(model, train_loader, loss_fn, optimizer):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(train_loader.dataset)

def test_loop(model, test_loader, loss_fn):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
    return test_loss / len(test_loader.dataset)

def predict_loop(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    predictions = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            predictions.append(output.cpu().numpy())
    return np.concatenate(predictions)

def train_network(model, train_loader, test_loader, loss_fn, optimizer, epochs):
    train_losses = []
    test_losses = []
    for epoch in range(epochs):
        train_loss = train_loop(model, train_loader, loss_fn, optimizer)
        test_loss = test_loop(model, test_loader, loss_fn)
        y_hat = predict_loop(model, test_loader)
        auc_roc = roc_auc_score(test_loader.dataset.targets, y_hat)
        auc_pr = average_precision_score(test_loader.dataset.targets, y_hat)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        print(f'Epoch: {epoch + 1}/{epochs} Train Loss: {train_loss:.4f} Test Loss: {test_loss:.4f} ROC AUC: {auc_roc:.4f} PR AUC: {auc_pr:.4f}')
    return train_losses, test_losses


def generate_dataset(X_train, y_train, n_examples, random_state,):

    train_examples = np.empty((n_examples, X_train.shape[1]))
    training_labels = []
    outlier_indices = np.where(y_train == 1)[0]
    inlier_indices = np.where(y_train == 0)[0]
    n_inliers = len(inlier_indices)
    n_outliers = len(outlier_indices)
    for i in range(n_examples):
        if(i % 2 == 0):
            sid = random_state.choice(n_inliers, 1)
            train_examples[i] = X_train[inlier_indices[sid]]
            training_labels += [0]
        else:
            sid = random_state.choice(n_outliers, 1)
            train_examples[i] = X_train[outlier_indices[sid]]
            training_labels += [1]

    training_labels = np.array(training_labels, dtype=np.float32)

    return train_examples, training_labels


Um conjunto de dados alternativo é criado para a rede ao amostrar dados anômalos e regulares de forma alternada.

In [81]:
column_transformer = compose_column_transformer(categorical_features, numerical_features)
X_train_transformed = column_transformer.fit_transform(X_train)
X_test_transformed = column_transformer.transform(X_test)

batch_size= 32

X_examples, y_examples = generate_dataset(X_train_transformed, y_train.values, n_examples=batch_size*1000, random_state=random_state)

train_dataset = CaseDataset(X_examples, y_examples)
test_dataset = CaseDataset(X_test_transformed, y_test.values)

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Treinamento da rede:

In [82]:
model = Net(X_train_transformed.shape[1])
loss_fn = DeviationLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
history = train_network(model, train_loader, test_loader, loss_fn, optimizer, epochs=10)

Epoch: 1/10 Train Loss: 0.0492 Test Loss: 0.0388 ROC AUC: 0.9324 PR AUC: 0.6818
Epoch: 2/10 Train Loss: 0.0300 Test Loss: 0.0368 ROC AUC: 0.9336 PR AUC: 0.6773
Epoch: 3/10 Train Loss: 0.0286 Test Loss: 0.0364 ROC AUC: 0.9362 PR AUC: 0.6835
Epoch: 4/10 Train Loss: 0.0277 Test Loss: 0.0360 ROC AUC: 0.9378 PR AUC: 0.6826
Epoch: 5/10 Train Loss: 0.0271 Test Loss: 0.0353 ROC AUC: 0.9390 PR AUC: 0.6782
Epoch: 6/10 Train Loss: 0.0265 Test Loss: 0.0346 ROC AUC: 0.9407 PR AUC: 0.6803
Epoch: 7/10 Train Loss: 0.0259 Test Loss: 0.0336 ROC AUC: 0.9428 PR AUC: 0.6785
Epoch: 8/10 Train Loss: 0.0253 Test Loss: 0.0324 ROC AUC: 0.9447 PR AUC: 0.6813
Epoch: 9/10 Train Loss: 0.0246 Test Loss: 0.0311 ROC AUC: 0.9454 PR AUC: 0.6775
Epoch: 10/10 Train Loss: 0.0237 Test Loss: 0.0297 ROC AUC: 0.9455 PR AUC: 0.6696


Curva de aprendizado da rede:

In [83]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(history[0])+1), y=history[0], name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(1, len(history[1])+1), y=history[1], name='Test Loss'))
fig.update_layout(title='Training History', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

Curva Precision-Recall:

In [84]:
y_hat = predict_loop(model, test_loader)
precision, recall, thresholds = precision_recall_curve(y_test, y_hat)
px.area(
    x=recall, 
    y=precision, 
    title=f'Precision-Recall Curve (AUC={auc(recall, precision):.2f})', 
    labels={'x': 'Recall', 'y': 'Precision'},
    )

Como a rede dá scores de anomalia, escolho um limiar de classificação que melhor equilibre a precisão e o recall. Num cenário real esse valor pode ser melhor ajustado para atender aos requisitos do problema.

In [85]:
fscore = (2 * precision * recall) / (precision*.2 + recall*.8)
fscore[np.isnan(fscore)] = 0
best_threshold = thresholds[np.argmax(fscore)]
log = model_evaluation(y_test, (y_hat > best_threshold).astype(int), 'NN (Deviation Loss)')
logbook.append(log)

AUC-ROC: 0.83
AUC-PR: 0.56
Precision: 0.73
Recall: 0.71
F1-Score: 0.72
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      2306
           1       0.73      0.71      0.72       478

    accuracy                           0.90      2784
   macro avg       0.83      0.83      0.83      2784
weighted avg       0.90      0.90      0.90      2784



C:\Users\jefma\AppData\Local\Temp\ipykernel_18872\2144431386.py:1: RuntimeWarning:

invalid value encountered in divide



O warning que aparece é causado por uma particularidade na forma que a função precision_recall_curve() retorna os dados.

## Comparação de resultados

In [86]:
results = pd.DataFrame(logbook)
results.style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\jefma\AppData\Local\Temp\ipykernel_18872\611257681.py:2: FutureWarning:

this method is deprecated in favour of `Styler.format(precision=..)`



,model-name,precision,recall,f1-score,auc-roc,auc-pr
0,Random Forest (Grid Search),0.80,0.86,0.83,0.91,0.71
1,NN (Deviation Loss),0.73,0.71,0.72,0.83,0.56


Para o conjunto de dados completo, o RandomForest se sai melhor que a Deviation Network. Quando se tem dados rotulados em quantidade suficiente para trabalhar com um modelo de classificação, um modelo de classificação deve ser priorizado. No entanto, existe um risco aqui. Se o padrão de fraudes muda ou fica pouco distinto do padrão de normalidade, o modelo de classificação pode encontrar dificuldades. Nesse tipo de problemas é comum que ocorra esse tipo de mudança e então o modelo de classificação perde desempenho de classificação por não conhecer aquelas características como anomalias. É aqui que um modelo próprio de detecção de fraudes é muito útil. No lugar de aprender a classificar uma credenciada, um modelo como a Deviation Network aprende o que é normal para uma credenciada e ao analisar uma credenciada com um novo padrão de fraude deve apontar aquilo como um desvio maior do que uma credenciada regular. O arquivo de resultados é o `results.csv`.

Para o conjunto de dados com as anomalias no treino reduzidas, o RandomForest tem uma boa precisão, mas um péssimo recall. Ao contrário, a Deviation Network tem um bom recall com uma boa precisão mesmo tendo apenas um conjunto muito reduzido de conhecimento prévio sobre as fraudes encontradas no conjunto. Uma outra vantagem aqui é a dimensão da rede que é bastante compacta. O aqruivo de resultados é o `results_60_fraudes.csv`.

In [87]:
results.to_csv('results.csv', index=False)

In [88]:
total_notebook_time = time.time() - notebook_start_time
print(f'Total runtime: {total_notebook_time/60:.2f} minutes')

Total runtime: 0.70 minutes


## BONUS SECTION ⭐

In [89]:
class DeviationLoss(nn.Module):
    def __init__(self,):
        super(DeviationLoss, self).__init__()

    def forward(self, outputs, targets):
        # Flatten
        outputs = outputs.view(-1)
        targets = targets.view(-1)

        confidence_margin = 5.
        # size=5000 is the setting of l in algorithm 1 in the paper
        ref = torch.from_numpy(np.random.normal(
            loc=0., scale=1.0, size=5000)).float()
        dev = (outputs - torch.mean(ref)) / torch.std(ref)
        inlier_loss = torch.abs(dev)
        outlier_loss = torch.abs(
            torch.max(confidence_margin - dev, torch.tensor(0.)))
        loss = torch.mean((1 - targets) * inlier_loss + targets * outlier_loss)
        return loss


class DevNetDataset(Dataset):
    def __init__(self, data: np.array, targets: np.array):
        self.x = data
        self.targets = targets

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        item = self.x[index, :]
        item = torch.from_numpy(item).float()
        targets = self.targets[index]
        targets = torch.from_numpy(np.array(targets)).float()
        return item, targets


class SpikingDevNet(nn.Module):
    def __init__(
            self,
            input_size: int,
            num_steps: int,
            spike_grad,
            beta: float = 0.9,
            learn_beta: bool = False,
            learn_threshold: bool = False,
            convert_to_spikes: bool = False,
            reset_mechanism: str = 'subtract'):
        super(SpikingDevNet, self).__init__()
        self.num_steps = num_steps
        self.convert_to_spikes = convert_to_spikes
        neuron_params = {
            'beta': beta,
            'spike_grad': spike_grad,
            'learn_beta': learn_beta,
            'learn_threshold': learn_threshold,
            'init_hidden': True}
        self.devnet_stack = nn.Sequential(
            nn.Linear(input_size, 20),
            snn.Leaky(**neuron_params),
            nn.Linear(20, 1),
            snn.Leaky(**neuron_params, output=True,
                      reset_mechanism=reset_mechanism),
        )

    def forward(self, x):
        utils.reset(self.devnet_stack)
        # Record output layer
        spk_dec_rec = []
        mem_dec_rec = []

        if self.convert_to_spikes:
            x = spikegen.rate(x, num_steps=self.num_steps,
                              time_var_input=False)
            for step in range(self.num_steps):
                spk_dec, mem_dec = self.devnet_stack(x[step])
                spk_dec_rec.append(spk_dec)
                mem_dec_rec.append(mem_dec)

        else:
            for step in range(self.num_steps):
                spk_dec, mem_dec = self.devnet_stack(x)
                spk_dec_rec.append(spk_dec)
                mem_dec_rec.append(mem_dec)

        return torch.stack(spk_dec_rec, dim=0), torch.stack(mem_dec_rec, dim=0)


def devnet_train_loop(dataloader, model, loss_fn, optimizer, device, epoch, count_spikes=False,):
    size = len(dataloader.dataset)
    tepoch = tqdm(dataloader)
    # tepoch.set_description(f'Epoch {epoch}')
    losses = []
    for batch in tepoch:
        X, targets = batch
        tepoch.set_description(f'Epoch {epoch}')
        # Compute prediction and loss
        X = X.to(device)
        targets = targets.to(device)
        spk_rec, mem_rec = model(X)
        if count_spikes:
            # total_spikes has dim [batch_size, num_neurons]
            total_spikes = spk_rec.sum(dim=0)
            spike_rates = total_spikes / model.num_steps
            loss = loss_fn(spike_rates, targets)
        else:
            loss = loss_fn(mem_rec[-1], targets)
        # loss = 0
        # for step in range(model.num_steps):
        #     loss += loss_fn(mem_rec[step], X)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tepoch.set_postfix(loss=loss.item())
        losses.append(loss.item())


def devnet_test_loop(dataloader, model, loss_fn, device, count_spikes=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for batch, X in enumerate(dataloader):
            X = X.to(device)
            spk_rec, mem_rec = model(X)
            num_steps = spk_rec.shape[0]
            if count_spikes:
                # total_spikes has dim [batch_size, num_neurons]
                total_spikes = spk_rec.sum(dim=0)
                spike_rates = total_spikes / num_steps
                loss = loss_fn(spike_rates, X)
            else:
                loss = loss_fn(mem_rec[-1], X)
            test_loss += loss.item()
    test_loss /= num_batches
    print(f'Avg loss: {test_loss:>8f} \n')


def devnet_predict_loop(dataloader, model, device):
    mem_rec_list = []
    spk_rec_list = []
    with torch.no_grad():
        for _, (X, _) in enumerate(dataloader):
            X = X.to(device)
            spk_rec, mem_rec = model(X)
            mem_rec_list.append(mem_rec)
            spk_rec_list.append(spk_rec)
    return torch.cat(spk_rec_list, dim=1), torch.cat(mem_rec_list, dim=1)


In [90]:
from tqdm import tqdm
# START MODEL
learning_rate = 5e-4
batch_size = 32
epochs = 10
num_steps = 60
input_size = X_train_transformed.shape[1]
spike_grad = surrogate.fast_sigmoid()
loss_fn = DeviationLoss()
count_spikes = False
convert_to_spikes = False
reset_mechanism = 'none' if not count_spikes else 'subtract'
# reset_mechanism = 'subtract'

train_ds = DevNetDataset(X_examples, y_examples)
test_ds = DevNetDataset(X_test_transformed, y_test.values)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using {device} device')

model = SpikingDevNet(
    input_size,
    num_steps,
    spike_grad,
    beta=0.9,
    learn_beta=False,
    learn_threshold=False,
    convert_to_spikes=convert_to_spikes,
    reset_mechanism=reset_mechanism,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dl = DataLoader(
    train_ds, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(
    test_ds, batch_size=batch_size, shuffle=False)

for t in range(epochs):
    # print(f'Epoch {t+1}/{epochs}\n-----------------')
    devnet_train_loop(train_dl, model, loss_fn,
                        optimizer, device, epoch=t, count_spikes=count_spikes,)
    # test_loop(x_test_dl, model, loss_fn,
    #           device, count_spikes=count_spikes)
print('Training finished')

start_time = time.time()
spk, mem = devnet_predict_loop(test_dl, model, device)

if count_spikes:
    total_spikes = spk.sum(dim=0)
    spike_rates = total_spikes / num_steps
    y_pred = spike_rates
else:
    y_pred = mem[-1, :]
scores = y_pred.cpu().numpy()

Using cpu device


Epoch 9: 100%|██████████| 1000/1000 [00:33<00:00, 29.54it/s, loss=0.584]


Training finished


In [91]:
y_hat = y_pred.numpy()

precision, recall, thresholds = precision_recall_curve(y_test, y_hat)
px.area(
    x=recall, 
    y=precision, 
    title=f'Precision-Recall Curve (AUC={auc(recall, precision):.2f})', 
    labels={'x': 'Recall', 'y': 'Precision'},
    )

In [92]:
fscore = (2 * precision * recall) / (precision*.2 + recall*.8)
fscore[np.isnan(fscore)] = 0
best_threshold = thresholds[np.argmax(fscore)]
log = model_evaluation(y_test, (y_hat > best_threshold).astype(int), 'SNN (Deviation Loss)')
logbook.append(log)

AUC-ROC: 0.85
AUC-PR: 0.59
Precision: 0.72
Recall: 0.76
F1-Score: 0.74
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.94      0.94      2306
           1       0.72      0.76      0.74       478

    accuracy                           0.91      2784
   macro avg       0.84      0.85      0.84      2784
weighted avg       0.91      0.91      0.91      2784



C:\Users\jefma\AppData\Local\Temp\ipykernel_18872\1194014097.py:1: RuntimeWarning:

invalid value encountered in divide



In [93]:
results = pd.DataFrame(logbook)
results.style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\jefma\AppData\Local\Temp\ipykernel_18872\611257681.py:2: FutureWarning:

this method is deprecated in favour of `Styler.format(precision=..)`



,model-name,precision,recall,f1-score,auc-roc,auc-pr
0,Random Forest (Grid Search),0.80,0.86,0.83,0.91,0.71
1,NN (Deviation Loss),0.73,0.71,0.72,0.83,0.56
2,SNN (Deviation Loss),0.72,0.76,0.74,0.85,0.59
